In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf 
import tensorflow_datasets as tfds
import os


import sys
sys.path.append('/data/codes/ma-fl-mia/')
from src import utils
from src import models
from src import data_preparation

conf = utils.load_config('/data/codes/ma-fl-mia/env.json', '/data/codes/ma-fl-mia/config.json')

In [5]:
train_ds, test_ds = tfds.load('cifar10', split=['train','test'], as_supervised=True)
X_train, Y_train = utils.get_np_from_tfds(train_ds)
conf['len_total_data'] = len(X_train)
X_split, Y_split = data_preparation.split_data(X_train, Y_train, conf['num_clients'], split_mode=conf['split_mode'],
                                               mode="clients", seed=conf['seed'], dirichlet_alpha=conf['dirichlet_alpha'])

In [16]:
cid = 6

x_train = X_split[cid]
y_train = Y_split[cid]

In [17]:
root_path = '/data/models/ma-fl-mia/federated/'
model_id = '20230510-112534'
epoch_id = 'saved_model_post_70'

In [18]:
import json
with open(os.path.join(root_path,model_id,"config.json")) as json_file:
    m_conf =  json.load(json_file)

In [19]:
m_path = os.path.join(root_path,model_id,'clients',str(cid),epoch_id)

model = models.get_model(unit_size=m_conf['unit_size']-10, conf=conf)
model.load_weights(os.path.join(m_path))
model.compile(optimizer=models.get_optimizer(),
              loss=models.get_loss(),
              metrics=["accuracy"])

In [20]:
from src import metrics

train_c_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))
r = metrics.evaluate(m_conf, model, train_c_ds, None, test_ds)

4/4 [==============================] - 0s 44ms/step


100%|██████████| 100/100 [00:00<00:00, 421.67it/s]

1/4 [======>.......................] - ETA: 0s

4/4 [==============================] - 0s 3ms/step


100%|██████████| 100/100 [00:00<00:00, 394.71it/s]


313/313 [==============================] - 1s 3ms/step


100%|██████████| 10000/10000 [00:22<00:00, 435.85it/s]


In [21]:
r

{'test_acc': 0.7093999981880188,
 'val_acc': None,
 'train_acc': 0.8650000095367432,
 'unit_size': 54,
 'alpha': 1.0,
 'model_id': '20230510-112534',
 'params': 739826,
 'model_mode': 'diao_CNN',
 'scale_mode': -10,
 'ma_mode': 'rm-cid',
 'adv_std': 14.580000000000002,
 'adv_pow': 14.7,
 'adv_cls': 14.14}

In [22]:
m_path = os.path.join(root_path,model_id,"saved_model")

model_global = models.get_model(unit_size=m_conf['unit_size'], conf=conf)
model_global.load_weights(os.path.join(m_path))
model_global.compile(optimizer=models.get_optimizer(),
              loss=models.get_loss(),
              metrics=["accuracy"])

In [23]:
r = metrics.evaluate(m_conf, model_global, train_ds, None, test_ds)

4/4 [==============================] - 0s 47ms/step


100%|██████████| 100/100 [00:00<00:00, 409.47it/s]


4/4 [==============================] - 0s 4ms/step


100%|██████████| 100/100 [00:00<00:00, 363.72it/s]


313/313 [==============================] - 1s 4ms/step


100%|██████████| 10000/10000 [00:22<00:00, 441.10it/s]


In [24]:
r

{'test_acc': 0.7491000294685364,
 'val_acc': None,
 'train_acc': 0.800819993019104,
 'unit_size': 54,
 'alpha': 1.0,
 'model_id': '20230510-112534',
 'params': 1112086,
 'model_mode': 'diao_CNN',
 'scale_mode': -10,
 'ma_mode': 'rm-cid',
 'adv_std': 7.28,
 'adv_pow': 7.5,
 'adv_cls': 6.54}

In [26]:
root_path = '/data/models/ma-fl-mia/federated/'
model_id = '20230509-170226'
epoch_id = 'saved_model_post_70'

import json
with open(os.path.join(root_path,model_id,"config.json")) as json_file:
    m_conf =  json.load(json_file)

m_path = os.path.join(root_path,model_id,'clients',str(cid),epoch_id)

model = models.get_model(unit_size=m_conf['unit_size'], conf=conf)
model.load_weights(os.path.join(m_path))
model.compile(optimizer=models.get_optimizer(),
              loss=models.get_loss(),
              metrics=["accuracy"])

from src import metrics

train_c_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))
r = metrics.evaluate(m_conf, model, train_c_ds, None, test_ds)

print(r)

m_path = os.path.join(root_path,model_id,"saved_model")

model_global = models.get_model(unit_size=m_conf['unit_size'], conf=conf)
model_global.load_weights(os.path.join(m_path))
model_global.compile(optimizer=models.get_optimizer(),
              loss=models.get_loss(),
              metrics=["accuracy"])

r = metrics.evaluate(m_conf, model_global, train_ds, None, test_ds)

print(r)

4/4 [==============================] - 0s 3ms/step


100%|██████████| 100/100 [00:00<00:00, 444.45it/s]

4/4 [==============================] - 0s 4ms/step



100%|██████████| 100/100 [00:00<00:00, 427.11it/s]


313/313 [==============================] - 1s 3ms/step


...

4/4 [==============================] - 0s 3ms/step


100%|██████████| 100/100 [00:00<00:00, 443.03it/s]


313/313 [==============================] - 1s 3ms/step


100%|██████████| 10000/10000 [00:21<00:00, 458.85it/s]


{'test_acc': 0.7465999722480774, 'val_acc': None, 'train_acc': 0.7884399890899658, 'unit_size': 38, 'alpha': 1.0, 'model_id': '20230509-170226', 'params': 552758, 'model_mode': 'diao_CNN', 'scale_mode': 1.0, 'ma_mode': 'no', 'adv_std': 4.82, 'adv_pow': 5.56, 'adv_cls': 4.9399999999999995}
